In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision
import torchvision.datasets as dsets
from torch.utils.data import DataLoader

In [2]:
import visdom
# visdom server실행해야함
vls = visdom.Visdom()

Setting up a new session...


# text 출력

In [3]:
vls.text("Hello", env="main")
# vls.close(env="main") evn가 main인 창 다 닫기, defalut는 main

'window_394ea180d4a728'

# image 출력

In [4]:
# 1개
a = torch.rand(3, 200, 200)
vls.image(a)

# 여러개
b = torch.Tensor(3,3,28,28)
vls.images(b)

'window_394ea180e1472e'

# dataset 준비

In [5]:
mnist_data = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="cifar10", train = True, transform=transforms.ToTensor(), download=True)

Files already downloaded and verified


# CIFAR10

In [6]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vls.images(data[0], env="main")

torch.Size([3, 32, 32])


'window_394ea181675972'

# MNIST

In [7]:
data = mnist_data.__getitem__(0)
print(data[0].shape)
vls.images(data[0], env="main")

torch.Size([1, 28, 28])


'window_394ea18169b97a'

### data loader

In [8]:
data_loader = DataLoader(dataset = mnist_data, batch_size = 32, shuffle = False)
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vls.images(value)
    # 하나의 batch만 출력 후 종료
    break

torch.Size([32, 1, 28, 28])


# Line Plot

In [9]:
Y = torch.randn(5)
# x를 넣지않으면 X값은 무조건 0 ~ 1
plt = vls.line(Y = Y)

X = torch.Tensor([1,2,3,4,5])
plt = vls.line(Y=Y, X=X)

### Line update

In [10]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

# plt에 X_append, Y_append를 append방식으로 update
vls.line(Y=Y_append, X=X_append, win=plt, update="append") 

'window_394ea181763596'

### multi line
x와 y의 shape가 같아야한다.

In [11]:
num = torch.Tensor(list(range(0,10)))
num = num.view(-1, 1)
num = torch.cat((num, num),dim = 1)

plt = vls.line(Y = torch.randn(10, 2), X = num)

### Line info
- option을 dict형태로 줘서 사용할 수 있다.
- showlegend 옵션을 사용해 범례를 추가할 수 있고, legend를 list형태로 주어 각 line의 이름을 작성할 수 있다.

In [12]:
plt = vls.line(Y=Y, X=X, opts = dict(title = "Test", showlegend = True))

plt = vls.line(Y=Y, X=X, opts = dict(title = "Test", legend = ["1번"], showlegend = True))

plt = vls.line(Y=torch.randn(10, 2), X = num, opts = dict(title = "Test", legend = ["1번", "2번"], showlegend = True))

# make function (update line)
- num과 loss_value는 Tensor이다.
- 다음과 같이 함수를 작성해 실시간으로 loss 그래프를 확인할 수 있다.

In [13]:
def loss_tracker(loss_plot, loss_value, num):
    vls.line(X = num, Y = loss_value, win = loss_plot, update = "append")
    
plt = vls.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [14]:
vls.close(env="main")

''